In [1]:
import pandas as pd
df = pd.read_csv("../../data/processed/processed_data_turkey_1900_2024_datetime.csv")
df

,Event ID,Datetime,Latitude,Longitude,Depth(km),xM,MD,ML,Mw,Ms,Mb,Type,Location
0,19000920000001,20/09/1900 00:00:01,37.8000,29.1000,5.0,5.0,5.0,-,NaN,-,-,Ke,DENIZLI (DENIZLI) [North East 2.3 km]
1,19010223000000,23/02/1901 00:00:00,37.9000,27.9000,15.0,4.8,4.7,4.6,4.8,4.6,4.7,Ke,KENGER- (AYDIN) [North East 1.1 km]
2,19010301000001,01/03/1901 00:00:01,38.2000,27.7000,5.0,5.0,5.0,-,NaN,-,-,Ke,YAKACIK-BAYINDIR (IZMIR) [South West 0.8 km]
3,19010401000001,01/04/1901 00:00:01,38.4000,31.4000,5.0,5.0,5.0,-,NaN,-,-,Ke,ATAKENT-AKSEHIR (KONYA) [North East 2.4 km]
4,19010501000001,01/05/1901 00:00:01,37.8000,27.8000,15.0,5.0,5.0,-,NaN,-,-,Ke,KADIKOY- (AYDIN) [South West 2.6 km]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64134,20240428211453,28/04/2024 21:14:53,38.5330,38.0423,5.4,3.9,-,3.8,3.9,-,-,Ke,CIVRIL-YAZIHAN (MALATYA) [South 3.0 km]
64135,20240428213328,28/04/2024 21:33:28,37.0177,29.7712,6.2,4.0,-,4.0,3.9,-,-,Ke,YAZIR-CAVDIR (BURDUR) [South East 2.5 km]
64136,20240429220835,29/04/2024 22:08:35,38.3103,38.2865,5.0,3.2,-,3.2,3.0,-,-,Ke,BOSTANBASI-YESILYURT (MALATYA) [East 3.1 km]
64137,20240430080325,30/04/2024 08:03:25,38.2458,31.5638,5.0,3.6,-,3.5,3.6,-,-,Ke,YAYLABELEN-AKSEHIR (KONYA) [North East 2.6 km]


In [2]:
def get_value_from_eid(df, event_id, column_name):
    # FIXME: Later add try exception block.
    value = df.loc[df['Event ID'] == event_id, column_name].values[0]
    return value

In [3]:
get_value_from_eid(df, 19000920000001, "xM")

5.0

In [4]:
import numpy as np


def haversine(lat1: float, lon1: float, lat2: float, lon2: float, r = 6371.0):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Applying haversine formula
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = np.sin(delta_lat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_lon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # Calculate the distance
    distance = c * r
    return distance

In [5]:
from datetime import datetime


def date_time_diff_in_hours(datetime_str_1, datetime_str_2):
    datetime1 = datetime.strptime(datetime_str_1, "%d/%m/%Y %H:%M:%S")
    datetime2 = datetime.strptime(datetime_str_2, "%d/%m/%Y %H:%M:%S")
    time_diff_h = abs((datetime1 - datetime2).total_seconds()) / 3600.
    return time_diff_h

(date_time_diff_in_hours("20/09/1900 00:00:01", "23/02/1901 00:00:00"),
date_time_diff_in_hours("23/02/1901 00:00:00", "20/09/1900 00:00:01"))

(3743.9997222222223, 3743.9997222222223)

## Metric Function & Optimization
### Implementing metric function V1

In [6]:
def earthquake_metric_function(df, eq1_event_id, eq2_event_id, W_mag, W_depth, W_distance, W_time):
    lat1 = get_value_from_eid(df, eq1_event_id, "Latitude")
    lat2 = get_value_from_eid(df, eq2_event_id, "Latitude")
    lon1 = get_value_from_eid(df, eq1_event_id, "Longitude")
    lon2 = get_value_from_eid(df, eq2_event_id, "Longitude")
    
    mag_diff = abs(get_value_from_eid(df, eq1_event_id, "xM") - get_value_from_eid(df, eq2_event_id, "xM"))
    depth_diff = abs(get_value_from_eid(df, eq1_event_id, "Depth(km)") - get_value_from_eid(df, eq2_event_id, "Depth(km)"))
    distance_diff = haversine(lat1, lon1, lat2, lon2)
    time_diff_hours = date_time_diff_in_hours(get_value_from_eid(df, eq1_event_id, "Datetime"), 
                                              get_value_from_eid(df, eq2_event_id, "Datetime"))
    
    weighted_sum = ((10 ** mag_diff) * W_mag + 
                    depth_diff * W_depth + 
                    distance_diff * W_distance +
                    time_diff_hours * W_time)
    return weighted_sum

### Optimizing Metric Function V1 (Iterative version)

In [16]:
def optimize_metric(weights, df):
    total_metric_value = 0
    W_mag, W_depth, W_distance, W_time = weights
    for i in range(len(df)):
        for j in range(i + 1, len(df)):
            # calculating metric value between for each (df[i], df[j]) pair
            metric_value = earthquake_metric_function(df, df.iloc[i]['Event ID'], df.iloc[j]['Event ID'], 
                                                      W_mag, W_depth, W_distance, W_time)
            total_metric_value += metric_value
    
    # To minimize metric value multipy total metric value with -1
    return -total_metric_value

### Apply bayesian optimization using skopt library

In [15]:
# TODO: Apply bayesian optimization.

Since our dataset contains 64k earthquakes iterative version is too slow so we need to optimize it.

### Implementing Metric Function V2 (Vectorized version)

In [7]:
# W_mag, W_depth, W_distance, W_time
def vectorized_earthquake_metric_function(mag_diff, depth_diff, distance_diff, time_diff, weights):
    return (weights[0] * (10 ** mag_diff) + 
            weights[1] * depth_diff + 
            weights[2] * distance_diff +
            weights[3] * time_diff)

### Implementing optimizer function (Vectorized Version)

In [8]:
# Optimizer Function (Vectorized)
def optimize_metric_vectorized(weights, df):
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy = df.copy()
    
    W_mag, W_depth, W_distance, W_time = weights  # Unpack the weights list
    weights = np.array([W_mag, W_depth, W_distance, W_time])
    
    # Extract the relevant columns as NumPy arrays
    magnitudes = df_copy['xM'].values
    depths = df_copy['Depth(km)'].values
    latitudes = df_copy['Latitude'].values
    longitudes = df_copy['Longitude'].values
    date_times = pd.to_datetime(df_copy['Datetime'], format="%d/%m/%Y %H:%M:%S").values
    
    # Calculate pairwise differences using broadcasting
    mag_diff = np.abs(magnitudes[:, None] - magnitudes)
    depth_diff = np.abs(depths[:, None] - depths)
    
    # Calculate pairwise haversine distances
    lat_diff = np.radians(latitudes[:, None] - latitudes)
    lon_diff = np.radians(longitudes[:, None] - longitudes)
    a = np.sin(lat_diff / 2) ** 2 + np.cos(np.radians(latitudes))[:, None] * np.cos(np.radians(latitudes)) * np.sin(lon_diff / 2) ** 2
    distance_diff = 2 * 6371.0 * np.arcsin(np.sqrt(a))
    
    # Calculate pairwise time differences in hours
    time_diff = np.abs((date_times[:, None] - date_times).astype('timedelta64[h]').astype(float))
    
    # Calculate the metric for all pairs
    total_metric_value = vectorized_earthquake_metric_function(mag_diff, depth_diff, distance_diff, time_diff, weights).sum()
    
    # Return the negative total metric value to minimize it
    return -total_metric_value

### Apply bayesian optimization using skopt library

In [16]:
# TODO: Apply bayesian optimization